In [ ]:
import numpy as np
import uproot as uproot
import uproot3 as uproot3
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from tqdm.notebook import tqdm

import pickle

In [ ]:
# this is currently unused, just here to explore the possibility of adding more fake data tests in the future


In [ ]:
# note that this fake data has to use PF files to get E_avail
# London used non-PF files for the prediction and systematics I'm using, but this is a small change that shouldn't really matter


In [ ]:
# TODO: generate 3D cross sections of P_mu, cos(theta_mu), and E_avail
# For the uboonetune as a reference, and for different generators with different M_A or z-expansion values
# Reweighting isn't perfect, but this should basically cover the information we're using, and should serve as
# an additional robust set of fake data tests with a more diverse set of nuclear treatments and M_A spreads

uboone_xs = np.ones((100, 100, 100)) # to be calculated

gibuu_MA_1_3_xs = np.ones((100, 100, 100)) # to be calculated


In [ ]:
bdt_vars = [
    "nue_score",
    "numu_score",
    "numu_cc_flag"
]

eval_vars = [
    "run",
    "subrun",
    "event",
    "truth_nuEnergy",
    "truth_nuPdg",
    "truth_isCC",
    "truth_vtxInside",
    "match_isFC",
    "match_completeness_energy",
    "truth_energyInside",

    "weight_cv",
    "weight_spline",
]

kine_vars = [
    "kine_reco_Enu",
]

pf_vars = [
    "reco_muonMomentum",
    "truth_muonMomentum",

    "truth_Ntrack",
    "truth_pdg",
    "truth_mother",
    "truth_vtxX",
    "truth_vtxY",
    "truth_vtxZ",
    "truth_startXYZT",
    "truth_startMomentum",
]


weight_vars = [
    # the framework never uses these, it uses the ones in T_eval instead!
    #"weight_cv",
    #"weight_spline",

    "All_UBGenie",
    
    "AxFFCCQEshape_UBGenie",
    "DecayAngMEC_UBGenie",
    "NormCCCOH_UBGenie",
    "NormNCCOH_UBGenie",
    "RPA_CCQE_UBGenie",
    "ThetaDelta2NRad_UBGenie",
    "Theta_Delta2Npi_UBGenie",
    "VecFFCCQEshape_UBGenie",
    "XSecShape_CCMEC_UBGenie",
    "xsr_scc_Fa3_SCC",
    "xsr_scc_Fv3_SCC",
]

loc = "/Users/leehagaman/data/processed_checkout_rootfiles/"
#loc = "/Users/leehagaman/data/from_london/"


In [ ]:
f = uproot3.open(loc + "checkout_prodgenie_bnb_nu_overlay_run1_PF.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nu_overlay_run1_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nu_overlay_run1_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nu_overlay_run1_df["file"] = "nu_overlay_run1"
del f
del f_bdt
del f_eval
del f_kine
del f_pfeval

print(nu_overlay_run1_df.shape)

f = uproot3.open(loc + "checkout_prodgenie_bnb_nu_overlay_run2_PF.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nu_overlay_run2_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nu_overlay_run2_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nu_overlay_run2_df["file"] = "nu_overlay_run2"
del f
del f_bdt
del f_eval
del f_kine
del f_pfeval

print(nu_overlay_run2_df.shape)

f = uproot3.open(loc + "checkout_prodgenie_bnb_nu_overlay_run3_PF.root")["wcpselection"]
f_bdt = f["T_BDTvars"].pandas.df(bdt_vars, flatten=False)
f_eval = f["T_eval"].pandas.df(eval_vars, flatten=False)
f_kine = f["T_KINEvars"].pandas.df(kine_vars, flatten=False)
f_pfeval = f["T_PFeval"].pandas.df(pf_vars, flatten=False)
nu_overlay_run3_pot = np.sum(f["T_pot"].pandas.df("pot_tor875good", flatten=False)["pot_tor875good"].to_numpy())
nu_overlay_run3_df = pd.concat([f_bdt, f_eval, f_kine, f_pfeval], axis=1, sort=False)
nu_overlay_run3_df["file"] = "nu_overlay_run3"
del f
del f_bdt
del f_eval
del f_kine
del f_pfeval

print(nu_overlay_run3_df.shape)

all_df = pd.concat([
    nu_overlay_run1_df, 
    nu_overlay_run2_df, 
    nu_overlay_run3_df], sort=False)

print(all_df.shape)


In [ ]:
data_pots = [
    1.42319e+20,
    2.5413e+20,
    2.40466e+20
]

ext_pots = [
    2.21814e+20,
    6.25014e+20,
    7.4127e+20,
]

weight_cv_vals = all_df["weight_cv"].to_numpy()
weight_spline_vals = all_df["weight_spline"].to_numpy()
files = all_df["file"].to_numpy()
net_weight_vals = []
for i in range(len(weight_cv_vals)):
    w_cv = weight_cv_vals[i]
    if not (0 < w_cv < 30):
        w_cv = 1
    
    if files[i] == "nu_overlay_run1":
        net_weight_vals.append(w_cv * weight_spline_vals[i] * data_pots[0] / nu_overlay_run1_pot)
    elif files[i] == "nu_overlay_run2":
        net_weight_vals.append(w_cv * weight_spline_vals[i] * data_pots[1] / nu_overlay_run2_pot)
    elif files[i] == "nu_overlay_run3":
        net_weight_vals.append(w_cv * weight_spline_vals[i] * data_pots[2] / nu_overlay_run3_pot)
    
all_df["net_weight"] = net_weight_vals


In [ ]:
# from /exp/uboone/app/users/bbogart/wcb-uboone-bdt

"""
double LEEana::get_true_Eavail(EvalInfo& eval, PFevalInfo& pfeval, bool useThreshold, bool useMass){
  double Eavail=0;
  //std::cout<<pfeval.truth_Ntrack;
  for(size_t i=0; i<pfeval.truth_Ntrack; i++){
    int pdgcode = pfeval.truth_pdg[i];
    int mother = pfeval.truth_mother[i];
    //if (mother!=0) { break; }
    if(mother==0){
      double dx = eval.truth_vtxX-pfeval.truth_startXYZT[i][0];
      double dy = eval.truth_vtxY-pfeval.truth_startXYZT[i][1];
      double dz = eval.truth_vtxZ-pfeval.truth_startXYZT[i][2];
      double vtx_diff = sqrt( dx*dx + dy*dy + dz*dz );
      if(vtx_diff>13.25){ continue; }//catches cases with two nu
      if( (eval.truth_nuTime*1000-pfeval.truth_startXYZT[i][3])>0.01){ continue; }//catches cases with two n

      if( abs(pdgcode)==2212 && 1*(pfeval.truth_startMomentum[i][3]*1000-938.27208816)<35 && useThreshold) continue;//below threshold porton, reject if using threshold
      else if( abs(pdgcode)==211 && pfeval.truth_startMomentum[i][3]*1000-139.57039<10 && useThreshold) continue;//below threshold pion, reject if using threshold
      else if( abs(pdgcode)==2212 ) Eavail+=1*(pfeval.truth_startMomentum[i][3]*1000-938.27208816)+useMass*8.36; //proton KE only
      else if(abs(pdgcode)==2112 ) continue; //primary neutron
      else if( abs(pdgcode)==13 && abs(eval.truth_nuPdg)==14 ) continue; //primary muon from numu
      else if( abs(pdgcode)==11 && abs(eval.truth_nuPdg)==12 ) continue; //primary e from nue
      else if( abs(pdgcode)==15 && abs(eval.truth_nuPdg)==16 ) continue; //primary tau from nutau
      else if( abs(pdgcode)==12 || abs(pdgcode)==14 || abs(pdgcode)==16 ) continue; //primary nu
      else if( abs(pdgcode)>3000 && abs(pdgcode)<4000 ) continue; //strange baryon
      else if( abs(pdgcode)==211 ) Eavail+=pfeval.truth_startMomentum[i][3]*1000-139.57039+useMass*139.57039;//pion, KE only
      else if (abs(pdgcode)<10000 ) Eavail+=pfeval.truth_startMomentum[i][3]*1000;//everything else add all energy
    }
  }
  //std::cout<<"  "<<Eavail<<std::endl; 
  return Eavail;
}
"""

# these seem to be the default settings in Ben's configuration files
useThreshold = False
useMass = False

truth_nuPdg_arr = all_df["truth_nuPdg"].to_numpy()
truth_Ntrack_arr = all_df["truth_Ntrack"].to_numpy()

truth_pdgs_arr = all_df["truth_pdg"].to_numpy()
truth_mothers_arr = all_df["truth_mother"].to_numpy()
truth_vtxXs_arr = all_df["truth_vtxX"].to_numpy()
truth_vtxYs_arr = all_df["truth_vtxY"].to_numpy()
truth_vtxZs_arr = all_df["truth_vtxZ"].to_numpy()
truth_startXYZTs_arr = all_df["truth_startXYZT"].to_numpy()
truth_startMomentums_arr = all_df["truth_startMomentum"].to_numpy()

E_avail_arr = []

num_events = len(truth_Ntrack_arr)
for i in tqdm(range(num_events)):

    truth_nuPdg = truth_nuPdg_arr[i]
    truth_Ntrack = truth_Ntrack_arr[i]

    truth_pdgs = truth_pdgs_arr[i]
    truth_mothers = truth_mothers_arr[i]
    truth_vtxX = truth_vtxXs_arr[i]
    truth_vtxY = truth_vtxYs_arr[i]
    truth_vtxZ = truth_vtxZs_arr[i]
    truth_startXYZTs = truth_startXYZTs_arr[i]
    truth_startMomentums = truth_startMomentums_arr[i]

    Eavail = 0

    for j in range(truth_Ntrack):
        truth_pdg = truth_pdgs[j]
        truth_mother = truth_mothers[j]
        truth_startXYZT = truth_startXYZTs[j]
        truth_startMomentum = truth_startMomentums[j]

        if truth_mother != 0:
            continue
        
        if abs(truth_pdg) == 2212: # true proton
            if abs(truth_startMomentum[3] * 1000 - 938.27208816) < 35 and useThreshold: # below 35 MeV, continue
                continue
            Eavail += truth_startMomentum[3] * 1000 - 938.27208816 + useMass * 8.36
        
        elif abs(truth_pdg) == 211: # true charged pion
            if abs(truth_startMomentum[3] * 1000 - 139.57039) < 10 and useThreshold: # below 10 MeV, continue
                continue
            Eavail += truth_startMomentum[3] * 1000 - 139.57039 + useMass * 139.57039

        elif abs(truth_pdg) == 2112: # true neutron
            continue
        
        elif abs(truth_pdg) == 13 and abs(truth_nuPdg) == 14: # true muon from numu
            continue

        elif abs(truth_pdg) == 11 and abs(truth_nuPdg) == 12: # true electron from nue
            continue

        elif abs(truth_pdg) == 15 and abs(truth_nuPdg) == 16: # true tau from nutau
            continue

        elif abs(truth_pdg) == 12 or abs(truth_pdg) == 14 or abs(truth_pdg) == 16: # true neutrino
            continue

        elif 3000 < abs(truth_pdg) < 4000: # strange baryon
            continue

        elif abs(truth_pdg) < 10000: # everything else
            Eavail += truth_startMomentum[3] * 1000

    E_avail_arr.append(Eavail)

all_df["E_avail"] = E_avail_arr
